In [24]:
!nvidia-smi

Thu Oct  5 03:01:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [25]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

# !pip -q install git+https://github.com/huggingface/transformers # need to install from github
# !pip install -q datasets loralib sentencepiece
# !pip -q install bitsandbytes accelerate xformers einops
# !pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done


In [26]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


In [27]:
!sudo apt-get install poppler-utils
!pip install pypdf
!pip install  InstructorEmbedding

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [29]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
#from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data

In [30]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/sample.pdf")
docs = loader.load()
len(docs)

2

In [31]:
docs

[Document(page_content='Page Break\n06/10/2021\nRK356748Sample Inv oice\nBilling Information Shipping Information\nCompany\nNulla justoName\nMariette CockerName\nAleece Whatley\nAddr ess\n7792 Merchant D, 0 Tony Ave\nWilmington, No, 28405Addr ess\n7792 Merchant D, 0 Tony Ave\nWilmington, No, 28405\nPhone Number\n(123)  123- 1232\nEmail\ncdagostini2@ebay.com\nDescription Quantity Unit Price Total\nProduct/Service 1\n Sink\n 2\n 100\n $200\nProduct/Service 2\n Nest Smart Filter\n 1\n 150\n $150\nProduct/Service 3\n Labor Fee\n 1\n 50\n $50\nProduct/Service 4\n Service Fee\n 1\n 25\n $25\nTotal:5999\nCreate your own automated PDFs with  Jotform  PDF Editor - It’s free\nRemove Branding1', metadata={'source': '/content/sample.pdf', 'page': 0}),
 Document(page_content='Create your own automated PDFs with  Jotform  PDF Editor - It’s free\nRemove Branding2', metadata={'source': '/content/sample.pdf', 'page': 1})]

In [32]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [33]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

2

In [34]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 2.98 s, sys: 1.15 s, total: 4.13 s
Wall time: 10.7 s


# Llama 2 13B

In [35]:
# from huggingface_hub import notebook_login

# notebook_login()

In [36]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [19]:
# !pip install bitsandbytes accelerate

In [39]:
pip install protobuf

In [42]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
#                                           use_auth_token=True,)

# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
#                                             #  device_map='auto',
#                                              torch_dtype=torch.float16,
#                                              use_auth_token=True,
#                                             #  load_in_8bit=True,
#                                             #  load_in_4bit=True
#                                              )

AttributeError: ignored

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [ ]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

# Chat with PDF

In [ ]:
result = qa_chain("how many invoice items are listed in the invoce?")

In [ ]:
result = qa_chain("unit price of each product")

In [ ]:
result = qa_chain("rcan you convert the tabel details in understandable paragraph")

In [ ]:
result = qa_chain("what is tax ID for the seller?")

In [ ]:
result = qa_chain("great, let me know client info")

In [ ]:
result = qa_chain("please retrieve invoice date")

In [ ]:
result = qa_chain("please retrieve invoice date, and invoice number. im not interested in description, just give me these two values")